# Empirical Dsitribution Predictors

We may be losing a lot of information in the annotations by condensing them into a single number. Instead, we can train a model to predict the empirical distribution formed by the annotations over the answer choices by minimizing the cross-entropy between the predicted distributions and the empirical distributions. This is essentially softmax classification, but off-the-shelf implementations don't let you pass a distribution as a training label, so we have to roll out own in TensorFlow.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from ngram import *
from baselines import *

In [2]:
data_filename  = '../../data/v4_annotated/annotated_onion_layer_5_rows_0_to_5000_raters_20.csv'
d = load_cf_labels(data_filename)
d = tidy_labels(d)
d = d_b.dropna(subset=['attack'])
d = d.iloc[np.random.permutation(np.arange(d.shape[0]))]

NameError: name 'd_b' is not defined

In [ ]:
ngram_feature_pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range = (1,6), max_features = 5000)),
    ('tfidf', TfidfTransformer(sublinear_tf=True,norm='l2')),
])

# Fit Plurality

In [ ]:
labels = plurality(d['attack'])

data = get_labeled_comments(d, labels)
train, test = split(data, test_size = 0.2,)

y_train =train.ix[:, train.columns != 'x'].values[:,0]
y_test =test.ix[:, train.columns != 'x'].values[:,0]
y_train = np.array([y_train, 1- y_train]).T
y_test = np.array([y_test, 1- y_test]).T

ngram_feature_extractor = feature_pipeline.fit(train['x'])
X_train = ngram_feature_extractor.transform(train['x'])
X_test = ngram_feature_extractor.transform(test['x'])

ED_CLF(X_train,
        y_train,
        X_test,
        y_test,
        training_epochs = 300,
        batch_size = 200,
        display_step = 15)

# Fit Empirical Distribution

In [ ]:
labels = empirical_dist(d['attack'], w = 0.5)
data = get_labeled_comments(d, labels)
train, test = split(data, test_size = 0.2,)

y_train =train.ix[:, train.columns != 'x'].values
y_test =test.ix[:, train.columns != 'x'].values

ngram_feature_extractor = feature_pipeline.fit(train['x'])
X_train = ngram_feature_extractor.transform(train['x'])
X_test = ngram_feature_extractor.transform(test['x'])

ED_CLF(X_train,
        y_train,
        X_test,
        y_test,
        training_epochs = 300,
        batch_size = 200,
        display_step = 15)